In [36]:
import numpy as np
import pandas as pd
from copy import deepcopy
from collections import Counter

In [68]:
class basic_preprocessing:
    
    
    def __init__(self, game_result, unit_result, augment_result, champion_result, item_result):

        self.game_result = game_result
        self.unit_result = unit_result
        self.augment_result = augment_result
        self.champion_result = champion_result
        self.item_result = item_result
        self.num_games = self.game_result.shape[0]//8        
        self.after_join = pd.merge(self.unit_result, self.game_result[['Game_id', 'Player_id', 'Placement', 'augment1', 'augment2', 'augment3']], on=['Game_id', 'Player_id'])
        self.after_join = self.after_join[['Game_id', 'Player_id', 'Placement', 'augment1', 'augment2', 'augment3', 'champ_name', 'rarity', 'tier', 'item1', 'item2', 'item3']]


    
    
    def change_gloves_item(self):
    
        check = np.logical_or(self.after_join.item1=='도적의 장갑', self.after_join.item1=='장난꾸러기의 장갑')

        self.after_join.loc[check, 'item2']='None'

        self.after_join.loc[check, 'item3'] = 'None'

        return self.after_join
    
    
    
    
    def check_item_unit_itemver(self):
    
        check = self.after_join.apply(lambda x : 3-list(x).count('None'), axis=1)


        self.after_join['num_item'] = check

        return self.after_join
    

    
    
    
    
    #basic preprocessing
    
    
    
    def make_augment_stack(self, game_result, augment_list, num_game):

        check = game_result[['Game_id', 'Player_id', 'Placement', 'augment1', 'augment2', 'augment3']].drop_duplicates()

        result = augment_list.copy()

        for i in range(1, 9):
            place = check.loc[check.Placement==i, ]

            augments = place.augment1.tolist() + place.augment2.tolist() + place.augment3.tolist()

            augments = Counter(augments)

            for augment in augments.keys():

                number = augments[augment]

                if i==1:
                    augment_score = [number, number*i, number, number, number, number]

                elif i<=4:
                    augment_score = [number, number*i, 0, number, 0, number]

                else:
                    augment_score = [number, number*i, 0, 0, 0, 0]

                result.loc[result.이름 == augment, 
                                 ['Count', 'Ave_score', 'Win_rate', 'Save_rate', 'Win_rate_all', 'Save_rate_all']] +=augment_score


        result.Ave_count = round(result.Count/num_game, 1)

        result.Ave_score = round(result.Ave_score/result.Count, 1)

        result.Win_rate = round(result.Win_rate/result.Count*100, 1)

        result.Save_rate = round(result.Save_rate/result.Count*100, 1)

        result.Win_rate_all = round(result.Win_rate_all/(num_game*8)*100, 1)

        result.Save_rate_all = round(result.Save_rate_all/(num_game*8)*100, 1)


        return result


    
    
    def make_unit_stack(self, game_result, champion_list, num_game):

        result = champion_list.copy()

        for placement in range(1, 9):

            for tier in range(1, 4):

                for num_item in range(1, 4):


                    case = np.logical_and(np.logical_and(game_result.Placement == placement, game_result.tier==tier),
                                          game_result.num_item == num_item)

                    champions = game_result.loc[case, 'champ_name'].tolist()

                    champions = Counter(champions)


                    for champion in champions.keys():

                        number = champions[champion]

                        if placement==1:



                            score = [number, number*placement, number*tier, number, number, number, number, num_item*number]

                        elif placement<=4:

                            score = [number, number*placement, number*tier, 0, number, 0, number, num_item*number]

                        else:

                            score = [number, number*placement, number*tier, 0, 0, 0, 0, num_item*number]

                        if num_item==3:

                            score.append(number)
                        else:
                            score.append(0)


                        result.loc[result.Name == champion, ['Count', 'Ave_score', 'Ave_star', 'Win_rate',
                                                                          'Save_rate', 'Win_rate_all', 'Save_rate_all',
                                                                          'Num_items', 'Full_item_count']] += score


        result.Ave_count = round(result.Count/num_game, 1)

        result.Ave_score = round(result.Ave_score / result.Count, 1)

        result.Win_rate = round(result.Win_rate / result.Count*100, 1)

        result.Save_rate = round(result.Save_rate / result.Count*100, 1)

        result.Win_rate_all = round(result.Win_rate_all / (num_game*8)*100, 1)

        result.Save_rate_all = round(result.Save_rate_all / (num_game*8)*100, 1)

        result.Num_items = round(result.Num_items/result.Count, 1)

        result.Full_item_count = round(result.Full_item_count/result.Count*100, 1)



        return result
    
    
    
    
    
    
    
    def make_item_stack(self, game_result, item_list, num_game):
    
        result = item_list.copy()

        for placement in range(1, 9):

            check = game_result.loc[game_result.Placement==placement, :]

            items = check.item1.tolist() + check.item2.tolist() + check.item3.tolist()

            items = Counter(items)

            for item in items.keys():

                if item=='None':
                    continue

                else:

                    number = items[item]

                    if placement==1:
                        item_score = [number, number*placement, number, number, number, number]

                    elif placement<=4:
                        item_score = [number, number*placement, 0, number, 0, number]

                    else:
                        item_score = [number, number*placement, 0, 0, 0, 0]

                result.loc[result.Name==item, ['Count', 'Ave_score', 'Win_rate',
                                                     'Save_rate', 'Win_rate_all', 'Save_rate_all']] += item_score


        result.Ave_count = round(result.Count/num_game, 1)

        result.Ave_score = round(result.Ave_score/result.Count, 1)

        result.Win_rate = round(result.Win_rate/result.Count*100, 1)

        result.Save_rate = round(result.Save_rate/result.Count*100, 1)

        result.Win_rate_all = round(result.Win_rate_all/(num_game*8)*100, 1)

        result.Save_rate_all = round(result.Save_rate_all/(num_game*8)*100, 1)



        return result

    
    
    
    
    
    
    
    # 증강체 기준 비교하기
    
    
    def augment_augment_crosscheck(self, game_result, augment_result, augment, num_game):

        check = np.logical_or(np.logical_or(game_result.augment1==augment, game_result.augment2==augment), game_result.augment3==augment)

        check = game_result[check]

        check_augment = self.make_augment_stack(check, augment_result,num_game)

        return check_augment
    
    
    def augment_champion_crosscheck(self, game_result, champion_result, augment, num_game):

        check = np.logical_or(np.logical_or(game_result.augment1==augment, game_result.augment2==augment), game_result.augment3==augment)

        check = game_result[check]

        check_champion = self.make_unit_stack(check, champion_result, num_game)


        return check_champion
    
    
    
    def augment_item_crosscheck(self, game_result, item_result, augment, num_game):

        check = np.logical_or(np.logical_or(game_result.augment1==augment, game_result.augment2==augment), game_result.augment3==augment)

        check = game_result[check]

        check_item = self.make_item_stack(check, item_result, num_game)

        return check_item
    
    
    # 챔피언 기준 비교하기
    
    
    def champion_champion_crosscheck(self, game_result, champion_result, champion, num_game):
    
        check = game_result.loc[game_result.champ_name == champion, ['Game_id', 'Player_id']]

        check = pd.merge(game_result, check, on=['Game_id', 'Player_id'])

        check_champion = self.make_unit_stack(check, champion_result, num_game)

        return check_champion
    
    
    
    
    def champion_augment_crosscheck(self, game_result, augment_result, champion, num_game):

        check = game_result.champ_name==champion

        check = game_result[check]

        check_augment = self.make_augment_stack(check, augment_result, num_game)

        return check_augment
    
    
    def champion_item_crosscheck(self, game_result, item_result, champion, num_game):
    
        check = game_result.champ_name==champion

        check = game_result[check]

        check_item = self.make_item_stack(check, item_result, num_game)

        return check_item
    
    
    
    # 아이템 기준
    
    
    def item_item_crosscheck(self, game_result, item_result, item, num_game):
    
        check = np.logical_or(np.logical_or(game_result.item1==item, game_result.item2==item), game_result.item3==item)


        check = game_result[check]


        check_item = self.make_item_stack(check, item_result, num_game)

        return check_item

    
    
    def item_augment_crosscheck(self, game_result, augment_result, item, num_game):
    
        check = np.logical_or(np.logical_or(game_result.item1==item, game_result.item2==item), game_result.item3==item)


        check = game_result[check]

        check_augment = self.make_augment_stack(check, augment_result, num_game)

        return check_augment
    
    
    
    def item_champion_crosscheck(self, game_result, champion_result, item, num_game):
    
        check = np.logical_or(np.logical_or(game_result.item1==item, game_result.item2==item), game_result.item3==item)    

        check = game_result[check]

        check_champion = self.make_unit_stack(check, champion_result, num_game)

        return check_champion

In [69]:
example_result = pd.read_csv('example1.csv')
example_unit= pd.read_csv('example_unit.csv')


example_result = example_result.drop('Unnamed: 0', axis=1)
example_unit = example_unit.drop('Unnamed: 0', axis=1)

unit_result = pd.read_csv('champion_list.csv', encoding='cp949')
augment_result = pd.read_csv('augment_list.csv', encoding='cp949')
item_result = pd.read_csv('item_list.csv', encoding='cp949')